<img src=https://api-manager.universia.net/coreplatform-document-management/api/document-management/public/6ra7ymjkfi64845 width="300" align="left">
<br />

# Aprendizaje automático I
#### <font color=green>*Máster en Informática Industrial y Robótica*</font>

Daniel Vidal Soroa  
Juan Diego Peña 

## Clasificación de mangos en tres clases según su presencia para la exportación, comercio local o procesamiento industrial

#### Importación de las librerías:

In [1]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import scipy.stats as stats
from statsmodels.stats.multicomp import pairwise_tukeyhsd, MultiComparison
from sklearn.model_selection import cross_validate
from sklearn.linear_model import LogisticRegression,LogisticRegressionCV
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.utils import shuffle
from skimage.io import imread
from skimage.transform import resize
from sklearn.metrics import roc_curve, auc, plot_confusion_matrix, classification_report, multilabel_confusion_matrix
from sklearn.model_selection import train_test_split
from random import seed
from random import randint
from tensorflow import keras
from keras import layers, models, utils, datasets
from keras import utils
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import KFold, GridSearchCV

In [2]:
%store -r scoring X X_train X_val CV t t_val f n 

no stored variable or alias X


#### Validación cruzada con Logistic Regression

In [9]:
modelLR = LogisticRegression(penalty='none', solver = 'lbfgs', max_iter=10000, multi_class='multinomial')

scoresLR = cross_validate(modelLR, X, t, cv=CV, scoring=scoring,n_jobs=-1,error_score='raise')

print('Media del LR accuracy : ', np.mean(scoresLR['test_accuracy']))

Media del LR accuracy :  0.9624074074074074


In [17]:
df = pd.DataFrame.from_dict(scoresLR)
df.drop(df.iloc[:, 0:2], inplace = True, axis = 1)
df.loc[5]=[np.mean(scoresLR['test_recall_macro']),np.mean(scoresLR['test_precision_macro']),np.mean(scoresLR['test_f1_macro']),np.mean(scoresLR['test_accuracy']),np.mean(scoresLR['test_roc_auc_ovo'])]
df.index = ["CV 1","CV 2","CV 3","CV 4","CV 5","Average"]
df.to_excel("./informe/tables/LR.xlsx") 
print(df)


         test_recall_macro  test_precision_macro  test_f1_macro  \
CV 1              0.955556              0.956299       0.955669   
CV 2              0.971296              0.971279       0.971257   
CV 3              0.962037              0.961981       0.961998   
CV 4              0.966667              0.966924       0.966639   
CV 5              0.956481              0.956644       0.956537   
Average           0.962407              0.962626       0.962420   

         test_accuracy  test_roc_auc_ovo  
CV 1          0.955556          0.987233  
CV 2          0.971296          0.987653  
CV 3          0.962037          0.984311  
CV 4          0.966667          0.993832  
CV 5          0.956481          0.986417  
Average       0.962407          0.987889  


### Analisis hiper-parametros Logistic regression  

In [14]:
param_grid = {
    "solvers": ['newton-cg', 'lbfgs'], 
    "logistic__C": np.logspace(-4, 4, 4),
}
search = GridSearchCV(estimator = modelLR, param_grid =param_grid, n_jobs=-1,cv=2,scoring=scoring,verbose=5,return_train_score=False)

search_results = search.fit(X,t)


Fitting 2 folds for each of 2 candidates, totalling 4 fits
